In [39]:
import pandas as pd

## Import Library Penting

In [40]:
df = pd.read_csv('video1-el.csv')
df

,timestamps,TP9,AF7,AF8,TP10,Right AUX
0,1.761114e+09,53.711,-7.812,5.859,91.797,0.0
1,1.761114e+09,-136.719,-0.977,1.465,-118.652,0.0
2,1.761114e+09,-126.953,-1.953,-5.371,-159.180,0.0
3,1.761114e+09,-12.695,-12.695,-4.395,-54.688,0.0
4,1.761114e+09,94.727,-14.160,4.395,58.594,0.0
...,...,...,...,...,...,...
73195,1.761114e+09,-46.875,-33.691,-25.879,-66.406,0.0
73196,1.761114e+09,-6.348,-28.809,-35.156,-44.922,0.0
73197,1.761114e+09,-18.555,-33.691,-24.902,-59.570,0.0
73198,1.761114e+09,-63.477,-43.945,-20.996,-69.336,0.0


## Ubah Timestamps sesuai dengan yang dibutuhkan

In [41]:
df['timestamps'] = pd.to_datetime(df['timestamps'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Jakarta')
df['timestamps'] = df['timestamps'].dt.floor('S')
df['timestamps'] = pd.to_datetime(df['timestamps'], errors='coerce')
df

C:\Users\hp_53\AppData\Local\Temp\ipykernel_22044\1684463464.py:2: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df['timestamps'] = df['timestamps'].dt.floor('S')


,timestamps,TP9,AF7,AF8,TP10,Right AUX
0,2025-10-22 13:14:01+07:00,53.711,-7.812,5.859,91.797,0.0
1,2025-10-22 13:14:01+07:00,-136.719,-0.977,1.465,-118.652,0.0
2,2025-10-22 13:14:01+07:00,-126.953,-1.953,-5.371,-159.180,0.0
3,2025-10-22 13:14:01+07:00,-12.695,-12.695,-4.395,-54.688,0.0
4,2025-10-22 13:14:01+07:00,94.727,-14.160,4.395,58.594,0.0
...,...,...,...,...,...,...
73195,2025-10-22 13:18:47+07:00,-46.875,-33.691,-25.879,-66.406,0.0
73196,2025-10-22 13:18:47+07:00,-6.348,-28.809,-35.156,-44.922,0.0
73197,2025-10-22 13:18:47+07:00,-18.555,-33.691,-24.902,-59.570,0.0
73198,2025-10-22 13:18:47+07:00,-63.477,-43.945,-20.996,-69.336,0.0


## Cek Rentang Timestamps

In [42]:
def rentang(df):
    min_time = df['timestamps'].min()
    max_time = df['timestamps'].max()

    print("Rentang timestamps:")
    print("Dari :", min_time)
    print("Sampai:", max_time)

    rentang_waktu = max_time - min_time
    print("Durasi rentang:", rentang_waktu)

In [43]:
rentang(df)

Rentang timestamps:
Dari : 2025-10-22 13:14:01+07:00
Sampai: 2025-10-22 13:18:47+07:00
Durasi rentang: 0 days 00:04:46


## Video asli itu aslinya ada 4.02 menit, kita cut data nya

In [44]:
min_time = df['timestamps'].min()
max_time = df['timestamps'].max()

start_cut = min_time + pd.Timedelta(seconds=10)
end_cut = max_time - pd.Timedelta(seconds=32)

df = df[(df['timestamps'] > start_cut) & (df['timestamps'] < end_cut)]
df

,timestamps,TP9,AF7,AF8,TP10,Right AUX
2763,2025-10-22 13:14:12+07:00,-53.223,8.301,34.668,-53.223,0.0
2764,2025-10-22 13:14:12+07:00,61.035,12.207,42.480,40.527,0.0
2765,2025-10-22 13:14:12+07:00,115.234,10.254,44.434,120.605,0.0
2766,2025-10-22 13:14:12+07:00,25.391,8.301,32.715,55.176,0.0
2767,2025-10-22 13:14:12+07:00,-76.660,2.441,31.250,-57.617,0.0
...,...,...,...,...,...,...
64971,2025-10-22 13:18:14+07:00,-34.180,-28.320,-39.551,-63.477,0.0
64972,2025-10-22 13:18:14+07:00,-28.320,-29.297,-34.180,-58.105,0.0
64973,2025-10-22 13:18:14+07:00,-4.883,-26.367,-29.297,1.953,0.0
64974,2025-10-22 13:18:14+07:00,2.441,-20.020,-26.367,22.461,0.0


In [45]:
rentang(df)

Rentang timestamps:
Dari : 2025-10-22 13:14:12+07:00
Sampai: 2025-10-22 13:18:14+07:00
Durasi rentang: 0 days 00:04:02


## Labeling sesuai label yang didapatkan

0 -> Netral
1 -> Negatif
2 -> Positif

In [46]:
label_seq = "222222002221100000222221"
start_time = df['timestamps'].min()

# hitung durasi (dalam detik) sejak waktu awal
df['elapsed'] = (df['timestamps'] - start_time).dt.total_seconds()

# tentukan blok waktu per 10 detik
df['block'] = (df['elapsed'] // 10).astype(int)

# ubah label_seq jadi list angka
label_list = list(label_seq)

# assign label berdasarkan waktu
df['label'] = df['block'].apply(lambda x: int(label_list[x]) if x < len(label_list) else None)

# opsional: hapus kolom bantu
df = df.drop(columns=['elapsed', 'block'])
df

C:\Users\hp_53\AppData\Local\Temp\ipykernel_22044\2934571433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['elapsed'] = (df['timestamps'] - start_time).dt.total_seconds()
C:\Users\hp_53\AppData\Local\Temp\ipykernel_22044\2934571433.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['block'] = (df['elapsed'] // 10).astype(int)
C:\Users\hp_53\AppData\Local\Temp\ipykernel_22044\2934571433.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,timestamps,TP9,AF7,AF8,TP10,Right AUX,label
2763,2025-10-22 13:14:12+07:00,-53.223,8.301,34.668,-53.223,0.0,2.0
2764,2025-10-22 13:14:12+07:00,61.035,12.207,42.480,40.527,0.0,2.0
2765,2025-10-22 13:14:12+07:00,115.234,10.254,44.434,120.605,0.0,2.0
2766,2025-10-22 13:14:12+07:00,25.391,8.301,32.715,55.176,0.0,2.0
2767,2025-10-22 13:14:12+07:00,-76.660,2.441,31.250,-57.617,0.0,2.0
...,...,...,...,...,...,...,...
64971,2025-10-22 13:18:14+07:00,-34.180,-28.320,-39.551,-63.477,0.0,NaN
64972,2025-10-22 13:18:14+07:00,-28.320,-29.297,-34.180,-58.105,0.0,NaN
64973,2025-10-22 13:18:14+07:00,-4.883,-26.367,-29.297,1.953,0.0,NaN
64974,2025-10-22 13:18:14+07:00,2.441,-20.020,-26.367,22.461,0.0,NaN


## Masih ada rentang kosong, margin of error dari pengambilan data, jadi kita hapus row dengan timestamps yang di belakangnya

In [ ]:
rentang(df[df['label'].isna()])

Rentang timestamps:
Dari : 2025-10-22 13:18:12+07:00
Sampai: 2025-10-22 13:18:14+07:00
Durasi rentang: 0 days 00:00:02


In [48]:
df[df['label'].isna()]

,timestamps,TP9,AF7,AF8,TP10,Right AUX,label
64208,2025-10-22 13:18:12+07:00,16.113,-17.090,11.719,21.484,0.0,NaN
64209,2025-10-22 13:18:12+07:00,-7.324,-15.137,9.766,-8.301,0.0,NaN
64210,2025-10-22 13:18:12+07:00,0.000,-12.695,9.766,-39.062,0.0,NaN
64211,2025-10-22 13:18:12+07:00,3.418,-18.066,8.301,-16.602,0.0,NaN
64212,2025-10-22 13:18:12+07:00,-3.418,-18.555,21.484,31.250,0.0,NaN
...,...,...,...,...,...,...,...
64971,2025-10-22 13:18:14+07:00,-34.180,-28.320,-39.551,-63.477,0.0,NaN
64972,2025-10-22 13:18:14+07:00,-28.320,-29.297,-34.180,-58.105,0.0,NaN
64973,2025-10-22 13:18:14+07:00,-4.883,-26.367,-29.297,1.953,0.0,NaN
64974,2025-10-22 13:18:14+07:00,2.441,-20.020,-26.367,22.461,0.0,NaN


In [49]:
df = df.dropna(subset=['label'])
df

,timestamps,TP9,AF7,AF8,TP10,Right AUX,label
2763,2025-10-22 13:14:12+07:00,-53.223,8.301,34.668,-53.223,0.0,2.0
2764,2025-10-22 13:14:12+07:00,61.035,12.207,42.480,40.527,0.0,2.0
2765,2025-10-22 13:14:12+07:00,115.234,10.254,44.434,120.605,0.0,2.0
2766,2025-10-22 13:14:12+07:00,25.391,8.301,32.715,55.176,0.0,2.0
2767,2025-10-22 13:14:12+07:00,-76.660,2.441,31.250,-57.617,0.0,2.0
...,...,...,...,...,...,...,...
64203,2025-10-22 13:18:11+07:00,1.465,-16.602,31.250,53.223,0.0,1.0
64204,2025-10-22 13:18:11+07:00,-20.020,-13.184,27.832,8.301,0.0,1.0
64205,2025-10-22 13:18:11+07:00,-12.695,-9.277,21.973,15.625,0.0,1.0
64206,2025-10-22 13:18:11+07:00,-13.184,-11.719,14.648,45.410,0.0,1.0


In [50]:
df[df['label'].isna()]

,timestamps,TP9,AF7,AF8,TP10,Right AUX,label
